In [106]:
class Operad:
    def __init__(self):
        self.colors = {}
        self.operations = {}

    def add_color(self, name, prop={}):
        if t:=self.colors.get(name):
            self.colors[name] = (t, prop)
        else:
            self.colors[name] = prop
        return name, t

    def add_operation(self, name, prop={}):
        self.operations[name] = prop
        return name


class Tree:
    def __init__(self, operation, trunk, branches, operad):

        trunk, parent = operad.add_color(trunk, self)
        self.trunk = trunk
        if parent:
            parent.branches[trunk] = self
            self.depth = parent.depth + 1
        else:
            self.depth = 0
        
        self.branches = {
            operad.add_color(branch, self)[0]: uTree(branch, depth=self.depth) for branch in branches
        }
        self.node = operad.add_operation(operation, self)

    def print_edges(self):
        return "".join(
            [f"{str(self.trunk)}\n"]
            + [
                "\t" * (self.depth + 1) + f"{branch.print_edges()}\n"
                for branch in self.branches.values()
            ]
        )

    def print_nodes(self):
        return "".join(
            [f"{str(self.node)}\n"]
            + [
                "\t" * (self.depth + 1) + f"{branch.print_nodes()}\n"
                for branch in self.branches.values()
            ]
        )

    def __str__(self):
        return f"{self.node}({','.join(self.branches.keys())};{self.trunk})"
    def __repr__(self):
        return f"{self.node}({','.join(self.branches.keys())};{self.trunk})"

class uTree(Tree):
    def __init__(self, name, depth):
        self.trunk = name
        self.branches = {}
        self.node = None
        self.depth = depth
    


In [107]:
import re
def string_to_tree_space(string, operad):

    operads = []
    operations = string.split("|")
    regex = r"(.*)\((.*)\)"
    for operation in operations:
        match = re.match(regex, operation)
        if not match:
            raise RuntimeError(f"Operation not defined correctly {operation}")
        operation, parameters = match.groups()
        branches, trunk = parameters.split(";")
        operads.append(Tree(operation, trunk, branches.split(","), operad))
    return operads[0], operad


def tree_space_to_string(tree_space):
    _, operad = tree_space
    return "|".join(str(tree) for tree in operad.operations.values())

def _recursive_str(tree, s):
    s.append(str(tree))
    for branch in tree.branches.values():
        if not isinstance(branch, uTree):
            _recursive_str(branch, s)
    return s

def tree_to_string(tree):
    return "|".join(_recursive_str(tree, []))
    


# S = string_to_tree_space("o0w(1,2;0)", Operad())
# T = string_to_tree_space("o0b(b,c;a)", Operad())
S = string_to_tree_space("0W(1;0)|1W(2,3;1)", Operad())
T = string_to_tree_space("0B(b,d;a)|1B(c;b)|2B(e;d)", Operad())

print(tree_space_to_string(S))
print(tree_to_string(S[0]))
print(tree_space_to_string(T))
print(tree_to_string(T[0]))

# print(S[0].print_nodes())

0W(1;0)|1W(2,3;1)
0W(1;0)|1W(2,3;1)
0B(b,d;a)|1B(c;b)|2B(e;d)
0B(b,d;a)|1B(c;b)|2B(e;d)


In [123]:
from copy import deepcopy


class DualTree:
    def __init__(self, S, T):
        self.S_og_space = deepcopy(S)
        self.T_og_space = deepcopy(T)
        self.dual_tree = deepcopy(S[0])
        self.T = deepcopy(T[0])

        self.add_color_base(self.dual_tree, self.T.trunk)

    def add_color_edge(self, T, c, depth):
        if "-" in T.trunk:
            print(T.trunk)
        T.trunk = f"{c}-{T.trunk}"
        T.branches = self.rename_keys(T, c, True)
        T.depth += depth
        for Ti in T.branches.values():
            self.add_color_edge(Ti, c, depth)
        return T

    def rename_keys(self, S, c, inv=False):
        aux = {}
        for i, Si in S.branches.items():
            if not inv:
                aux[f"{i}-{c}"] = Si
            else:
                aux[f"{c}-{i}"] = Si
        return aux

    def add_color_base(self, S, c):
        S.trunk = f"{S.trunk}-{c}"

        S.branches = self.rename_keys(S, c)

        for i, Si in S.branches.items():
            if isinstance(Si, uTree):
                S.branches[i] = self.add_color_edge(deepcopy(self.T), i.split("-")[0], S.depth + 1)
            else:
                self.add_color_base(Si, c)


S_T = DualTree(S, T)
T_S = DualTree(T, S)
# print(DT.dual_tree.print_edges())
print(tree_to_string(S_T.dual_tree))
print(tree_to_string(T_S.dual_tree))


0W(1-a;0-a)|1W(2-a,3-a;1-a)|0B(2-b,2-d;2-a)|1B(2-c;2-b)|2B(2-e;2-d)|0B(3-b,3-d;3-a)|1B(3-c;3-b)|2B(3-e;3-d)
0B(b-0,d-0;a-0)|1B(c-0;b-0)|0W(c-1;c-0)|1W(c-2,c-3;c-1)|2B(e-0;d-0)|0W(e-1;e-0)|1W(e-2,e-3;e-1)
